# 🧫 🧪 Эксперименты с классическими *unsupervised* методами обнаружения аномалий

### 🌐 Установка [pyod](https://github.com/yzhao062/pyod)

In [1]:
# !pip3 install pyod

In [2]:
# import os 
# os.environ['MKL_NUM_THREADS'] = '12' 
# os.environ['GOTO_NUM_THREADS'] = '12' 
# os.environ['OMP_NUM_THREADS'] = '12' 
# os.environ['openmp'] = 'True'

## 💅 Предобработка данных

In [3]:
# !pip3 install pymorphy2

In [4]:
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_multiple_whitespaces

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


def preprocess_text(text, tokenization=False):
    """
    Предобрабатывает исходный русский текст.
    Выполняет очистку, токенизацию и лемматизацию.
    Возвращает список токенов или предобработанный текст.
    """
    text = text.lower()                     # приведение к нижнему регистру
    text = strip_tags(text)                 # удаление тэгов
    text = strip_numeric(text)              # удаление цифр
    text = strip_punctuation(text)          # удаление пунктуации
    text = strip_non_alphanum(text)         # удаление небуквенных символов
    text = strip_short(text)                # удаление коротких слов
    text = strip_multiple_whitespaces(text) # удаление множественных пробелов
    
    # Токенизация
    list_of_words = word_tokenize(text, language='russian') 

    # Удаление стоп-слов
    list_of_words = [w for w in list_of_words if w not in stopwords.words('russian')]

    # Лемматизация
    for i in range(len(list_of_words)): 
        list_of_words[i] = morph.parse(list_of_words[i])[0].normal_form

    if tokenization: # возвращаем список слов
        return list_of_words
    else: # возвращаем строку
        return ' '.join(w for w in list_of_words) 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pavelmamaev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pavelmamaev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 🧬 Экперименты

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json

# tf.config.threading.set_inter_op_parallelism_threads(6)

tf.keras.backend.set_floatx('float64')

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score

import pyod
from pyod.models import auto_encoder
from pyod.models import vae

c = 0.1  # отношение количества аномальных экземпляров к нормальным

# df = pd.read_csv("lenta-ru-news.csv")

# list_selecting_topics = [
#                          ('Наука и техника', 'Оружие'),
#                          ('Культура', 'Театр'),
#                          ('Силовые структуры', 'Полиция и спецслужбы'),
#                          ('Спорт', 'Бокс и ММА'),
#                          ('Наука и техника', 'Космос'),
#                          ('Наука и техника', 'Наука'),
#                          ('Мир', 'Политика'),
#                          ('Экономика', 'Госэкономика')
# ]

# topics_dict = {}
# for t in list_selecting_topics:
#     topic, tag = t[0], t[1]
#     tmp_df = df[df['topic']==topic]
#     tmp_df = tmp_df[tmp_df['tags']==tag]
#     topics_dict[topic + '-' + tag] = list(tmp_df['text'].reset_index(drop=True)[:1000])

# # Предобработка текста
# for t in topics_dict:
#     topics_dict[t] = [preprocess_text(text) for text in topics_dict[t]]
    
# with open('my_dict.json', 'w') as fp:
#     json.dump(topics_dict, fp)
    
# print('УСПЕХ!')


# Загрузка предобработанных текстов
with open('my_dict.json', 'r') as fp:
    topics_dict = json.load(fp)
    
experimant_cnt = 0
all_experiments = len(topics_dict) * (len(topics_dict) -  1)
auc_list_ae = []
auc_list_vae = []


# Перебираем пары категорий
for t1 in topics_dict:
    for t2 in topics_dict:
        if t1 == t2:
            continue

        experimant_cnt += 1
        
        if experimant_cnt <= 42:
            continue

        # Формирование нормальной и аномальной выборок
        normal_data = topics_dict[t1]
        anomal_data = topics_dict[t2][:min(int(c * len(normal_data)) + 1, len(topics_dict[t2]))]
        all_data = normal_data + anomal_data

        # TF-IDF векторизация
        vectorizer = TfidfVectorizer()
        all_data_tf = vectorizer.fit_transform(all_data).toarray()

        # Формирование выборок
        x = all_data_tf
        y = np.array([False] * len(normal_data) + [True] * len(anomal_data))
        all_data, x, y = shuffle(all_data, x, y, random_state=123)

        # Задаем модели

        # AE
        
        ae_clf = auto_encoder.AutoEncoder(
            hidden_neurons=[64, 32, 16, 32, 64],
            hidden_activation='relu',
            output_activation='sigmoid',
            optimizer='adam',
            epochs=1,
            batch_size=512,
            dropout_rate=0.3,
            l2_regularizer=0.4,
            validation_size=0.0,
            preprocessing=True,
            verbose=0,
            random_state=123,
            contamination=0.1
        )
        

        # VAE
        
        vae_clf = vae.VAE(
            contamination=0.1,
            encoder_neurons=[64, 32],
            decoder_neurons=[32, 64],
            latent_dim=5,
            hidden_activation='relu',
            output_activation='sigmoid',
            optimizer='adam',
            epochs=1,
            batch_size=512,
            dropout_rate=0.3,
            l2_regularizer=0.4,
            validation_size=0.0,
            preprocessing=True,
            verbose=0,
            random_state=123,
        )

        # Тренировка моделей
        try:
            ae_clf.fit(x)
            vae_clf.fit(x)
            
        except ValueError:
            continue
            

        # Предсказание моделей
        y_predict_ae = ae_clf.decision_function(x)
        y_predict_vae = vae_clf.decision_function(x)

        # Считаем метрику ROC AUC
        auc_ae = roc_auc_score(y, y_predict_ae)
        auc_vae = roc_auc_score(y, y_predict_vae)

        # Добавляем в списки
        auc_list_ae.append(auc_ae)
        auc_list_vae.append(auc_vae)
        

        # Вывод результатов
        print("-" * 50)
        print("Эксперимент №{}/{}  с normal = {}, anomal = {}".format(
            experimant_cnt, all_experiments, t1, t2))
        print("auc_ae = ", auc_ae)
        print("auc_vae = ", auc_vae)
        print("-" * 50)


auc_np = np.array(auc_list_ae)
print("*" * 50)
print("AE Медиана auc = {}".format(np.median(auc_np)))
print("AE Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

auc_np = np.array(auc_list_vae)
print("*" * 50)
print("VAE Медиана auc = {}".format(np.median(auc_np)))
print("VAE Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

--------------------------------------------------
Эксперимент №43/56  с normal = Мир-Политика, anomal = Наука и техника-Оружие
auc_ae =  0.7532376237623762
auc_vae =  0.7539306930693069
--------------------------------------------------
--------------------------------------------------
Эксперимент №44/56  с normal = Мир-Политика, anomal = Культура-Театр
auc_ae =  0.7721683168316831
auc_vae =  0.772128712871287
--------------------------------------------------
--------------------------------------------------
Эксперимент №45/56  с normal = Мир-Политика, anomal = Силовые структуры-Полиция и спецслужбы
auc_ae =  0.6600693069306931
auc_vae =  0.6598118811881187
--------------------------------------------------
--------------------------------------------------
Эксперимент №46/56  с normal = Мир-Политика, anomal = Спорт-Бокс и ММА
auc_ae =  0.3524059405940594
auc_vae =  0.35276237623762374
--------------------------------------------------
----------------------------------------------

--------------------------------------------------
Эксперимент №49/56  с normal = Мир-Политика, anomal = Экономика-Госэкономика
auc_ae =  0.7044554455445544
auc_vae =  0.7041485148514852
--------------------------------------------------
--------------------------------------------------
Эксперимент №50/56  с normal = Экономика-Госэкономика, anomal = Наука и техника-Оружие
auc_ae =  0.8247821782178218
auc_vae =  0.8247326732673268
--------------------------------------------------
--------------------------------------------------
Эксперимент №51/56  с normal = Экономика-Госэкономика, anomal = Культура-Театр
auc_ae =  0.8496237623762377
auc_vae =  0.8495940594059407
--------------------------------------------------
--------------------------------------------------
Эксперимент №52/56  с normal = Экономика-Госэкономика, anomal = Силовые структуры-Полиция и спецслужбы
auc_ae =  0.7615643564356437
auc_vae =  0.7616336633663368
--------------------------------------------------
----------

--------------------------------------------------
Эксперимент №54/56  с normal = Экономика-Госэкономика, anomal = Наука и техника-Космос
auc_ae =  0.7716336633663367
auc_vae =  0.7714752475247525
--------------------------------------------------
--------------------------------------------------
Эксперимент №55/56  с normal = Экономика-Госэкономика, anomal = Наука и техника-Наука
auc_ae =  0.9721386138613861
auc_vae =  0.9722475247524753
--------------------------------------------------
--------------------------------------------------
Эксперимент №56/56  с normal = Экономика-Госэкономика, anomal = Мир-Политика
auc_ae =  0.8626138613861386
auc_vae =  0.863
--------------------------------------------------
**************************************************
AE Медиана auc = 0.7665990099009902
AE Среднее auc = 0.7471937765205092
**************************************************
**************************************************
VAE Медиана auc = 0.7665544554455446
VAE Среднее auc = 

## 🏁 Результаты

--------------------------------------------------
Эксперимент №1/56  с normal = Наука и техника-Оружие, anomal = Культура-Театр
auc_ae =  0.903069306930693
auc_vae =  0.9030396039603961
--------------------------------------------------
--------------------------------------------------
Эксперимент №2/56  с normal = Наука и техника-Оружие, anomal = Силовые структуры-Полиция и спецслужбы
auc_ae =  0.8225445544554456
auc_vae =  0.8225049504950496
--------------------------------------------------
--------------------------------------------------
Эксперимент №3/56  с normal = Наука и техника-Оружие, anomal = Спорт-Бокс и ММА
auc_ae =  0.5366831683168317
auc_vae =  0.5369009900990099
--------------------------------------------------
--------------------------------------------------
Эксперимент №4/56  с normal = Наука и техника-Оружие, anomal = Наука и техника-Космос
auc_ae =  0.7370594059405942
auc_vae =  0.7380000000000001
--------------------------------------------------
--------------------------------------------------
Эксперимент №5/56  с normal = Наука и техника-Оружие, anomal = Наука и техника-Наука
auc_ae =  0.9738217821782178
auc_vae =  0.9738118811881188
--------------------------------------------------
--------------------------------------------------
Эксперимент №6/56  с normal = Наука и техника-Оружие, anomal = Мир-Политика
auc_ae =  0.9070396039603961
auc_vae =  0.907089108910891
--------------------------------------------------
--------------------------------------------------
Эксперимент №7/56  с normal = Наука и техника-Оружие, anomal = Экономика-Госэкономика
auc_ae =  0.8140792079207921
auc_vae =  0.814089108910891
--------------------------------------------------
--------------------------------------------------
Эксперимент №8/56  с normal = Культура-Театр, anomal = Наука и техника-Оружие
auc_ae =  0.6724257425742574
auc_vae =  0.6721683168316831
--------------------------------------------------
--------------------------------------------------
Эксперимент №9/56  с normal = Культура-Театр, anomal = Силовые структуры-Полиция и спецслужбы
auc_ae =  0.5414851485148515
auc_vae =  0.5413861386138614
--------------------------------------------------
--------------------------------------------------
Эксперимент №10/56  с normal = Культура-Театр, anomal = Спорт-Бокс и ММА
auc_ae =  0.2612970297029703
auc_vae =  0.2610891089108911
--------------------------------------------------
--------------------------------------------------
Эксперимент №11/56  с normal = Культура-Театр, anomal = Наука и техника-Космос
auc_ae =  0.630831683168317
auc_vae =  0.6314059405940595
--------------------------------------------------
--------------------------------------------------
Эксперимент №12/56  с normal = Культура-Театр, anomal = Наука и техника-Наука
auc_ae =  0.9162574257425743
auc_vae =  0.9163564356435644
--------------------------------------------------
--------------------------------------------------
Эксперимент №13/56  с normal = Культура-Театр, anomal = Мир-Политика
auc_ae =  0.7240792079207922
auc_vae =  0.7239207920792079
--------------------------------------------------
--------------------------------------------------
Эксперимент №14/56  с normal = Культура-Театр, anomal = Экономика-Госэкономика
auc_ae =  0.6346930693069308
auc_vae =  0.6345049504950495
--------------------------------------------------
--------------------------------------------------
Эксперимент №15/56  с normal = Силовые структуры-Полиция и спецслужбы, anomal = Наука и техника-Оружие
auc_ae =  0.7732178217821782
auc_vae =  0.7731980198019802
--------------------------------------------------
--------------------------------------------------
Эксперимент №16/56  с normal = Силовые структуры-Полиция и спецслужбы, anomal = Культура-Театр
auc_ae =  0.7854653465346534
auc_vae =  0.7859405940594059
--------------------------------------------------
--------------------------------------------------
Эксперимент №17/56  с normal = Силовые структуры-Полиция и спецслужбы, anomal = Спорт-Бокс и ММА
auc_ae =  0.4411881188118812
auc_vae =  0.44102970297029703
--------------------------------------------------
--------------------------------------------------
Эксперимент №18/56  с normal = Силовые структуры-Полиция и спецслужбы, anomal = Наука и техника-Космос
auc_ae =  0.7571782178217822
auc_vae =  0.7572772277227722
--------------------------------------------------
--------------------------------------------------
Эксперимент №19/56  с normal = Силовые структуры-Полиция и спецслужбы, anomal = Наука и техника-Наука
auc_ae =  0.9726237623762376
auc_vae =  0.9726336633663366
--------------------------------------------------
--------------------------------------------------
Эксперимент №20/56  с normal = Силовые структуры-Полиция и спецслужбы, anomal = Мир-Политика
auc_ae =  0.8266138613861386
auc_vae =  0.8266336633663367
--------------------------------------------------
--------------------------------------------------
Эксперимент №21/56  с normal = Силовые структуры-Полиция и спецслужбы, anomal = Экономика-Госэкономика
auc_ae =  0.7537425742574257
auc_vae =  0.7538712871287129
--------------------------------------------------
--------------------------------------------------
Эксперимент №22/56  с normal = Спорт-Бокс и ММА, anomal = Наука и техника-Оружие
auc_ae =  0.9394356435643565
auc_vae =  0.9395445544554456
--------------------------------------------------
--------------------------------------------------
Эксперимент №23/56  с normal = Спорт-Бокс и ММА, anomal = Культура-Театр
auc_ae =  0.9186732673267327
auc_vae =  0.9186534653465347
--------------------------------------------------
--------------------------------------------------
Эксперимент №24/56  с normal = Спорт-Бокс и ММА, anomal = Силовые структуры-Полиция и спецслужбы
auc_ae =  0.8899108910891089
auc_vae =  0.8899801980198021
--------------------------------------------------
--------------------------------------------------
Эксперимент №25/56  с normal = Спорт-Бокс и ММА, anomal = Наука и техника-Космос
auc_ae =  0.9007623762376238
auc_vae =  0.9007821782178218
--------------------------------------------------
--------------------------------------------------
Эксперимент №26/56  с normal = Спорт-Бокс и ММА, anomal = Наука и техника-Наука
auc_ae =  0.9926237623762376
auc_vae =  0.9926237623762377
--------------------------------------------------
--------------------------------------------------
Эксперимент №27/56  с normal = Спорт-Бокс и ММА, anomal = Мир-Политика
auc_ae =  0.953079207920792
auc_vae =  0.9530792079207921
--------------------------------------------------
--------------------------------------------------
Эксперимент №28/56  с normal = Спорт-Бокс и ММА, anomal = Экономика-Госэкономика
auc_ae =  0.9220693069306931
auc_vae =  0.9221881188118812
--------------------------------------------------
--------------------------------------------------
Эксперимент №29/56  с normal = Наука и техника-Космос, anomal = Наука и техника-Оружие
auc_ae =  0.7428019801980199
auc_vae =  0.7424653465346535
--------------------------------------------------
--------------------------------------------------
Эксперимент №30/56  с normal = Наука и техника-Космос, anomal = Культура-Театр
auc_ae =  0.8765544554455446
auc_vae =  0.8764455445544554
--------------------------------------------------
--------------------------------------------------
Эксперимент №31/56  с normal = Наука и техника-Космос, anomal = Силовые структуры-Полиция и спецслужбы
auc_ae =  0.8035346534653465
auc_vae =  0.8037425742574258
--------------------------------------------------
--------------------------------------------------
Эксперимент №32/56  с normal = Наука и техника-Космос, anomal = Спорт-Бокс и ММА
auc_ae =  0.5132277227722772
auc_vae =  0.5134158415841584
--------------------------------------------------
--------------------------------------------------
Эксперимент №33/56  с normal = Наука и техника-Космос, anomal = Наука и техника-Наука
auc_ae =  0.928009900990099
auc_vae =  0.9279999999999999
--------------------------------------------------
--------------------------------------------------
Эксперимент №34/56  с normal = Наука и техника-Космос, anomal = Мир-Политика
auc_ae =  0.8967722772277228
auc_vae =  0.8968118811881188
--------------------------------------------------
--------------------------------------------------
Эксперимент №35/56  с normal = Наука и техника-Космос, anomal = Экономика-Госэкономика
auc_ae =  0.7967227722772277
auc_vae =  0.7963465346534654
--------------------------------------------------
--------------------------------------------------
Эксперимент №36/56  с normal = Наука и техника-Наука, anomal = Наука и техника-Оружие
auc_ae =  0.4378910891089109
auc_vae =  0.4377029702970297
--------------------------------------------------
--------------------------------------------------
Эксперимент №37/56  с normal = Наука и техника-Наука, anomal = Культура-Театр
auc_ae =  0.6308712871287129
auc_vae =  0.6310990099009901
--------------------------------------------------
--------------------------------------------------
Эксперимент №38/56  с normal = Наука и техника-Наука, anomal = Силовые структуры-Полиция и спецслужбы
auc_ae =  0.4983960396039604
auc_vae =  0.4981881188118812
--------------------------------------------------

Эксперимент №39/56  с normal = Наука и техника-Наука, anomal = Спорт-Бокс и ММА
auc_ae =  0.17082178217821783
auc_vae =  0.16985148514851486
--------------------------------------------------
--------------------------------------------------
Эксперимент №40/56  с normal = Наука и техника-Наука, anomal = Наука и техника-Космос
auc_ae =  0.22625742574257426
auc_vae =  0.22586138613861387
--------------------------------------------------
--------------------------------------------------
Эксперимент №41/56  с normal = Наука и техника-Наука, anomal = Мир-Политика
auc_ae =  0.6673861386138614
auc_vae =  0.6683168316831682
--------------------------------------------------

--------------------------------------------------
Эксперимент №43/56  с normal = Мир-Политика, anomal = Наука и техника-Оружие
auc_ae =  0.7532376237623762
auc_vae =  0.7539306930693069
--------------------------------------------------
--------------------------------------------------
Эксперимент №44/56  с normal = Мир-Политика, anomal = Культура-Театр
auc_ae =  0.7721683168316831
auc_vae =  0.772128712871287
--------------------------------------------------
--------------------------------------------------
Эксперимент №45/56  с normal = Мир-Политика, anomal = Силовые структуры-Полиция и спецслужбы
auc_ae =  0.6600693069306931
auc_vae =  0.6598118811881187
--------------------------------------------------
--------------------------------------------------
Эксперимент №46/56  с normal = Мир-Политика, anomal = Спорт-Бокс и ММА
auc_ae =  0.3524059405940594
auc_vae =  0.35276237623762374
--------------------------------------------------
--------------------------------------------------
Эксперимент №47/56  с normal = Мир-Политика, anomal = Наука и техника-Космос
auc_ae =  0.7291980198019802
auc_vae =  0.7292079207920793
--------------------------------------------------
--------------------------------------------------
Эксперимент №48/56  с normal = Мир-Политика, anomal = Наука и техника-Наука
auc_ae =  0.9649009900990099
auc_vae =  0.9647821782178219
--------------------------------------------------
--------------------------------------------------
Эксперимент №49/56  с normal = Мир-Политика, anomal = Экономика-Госэкономика
auc_ae =  0.7044554455445544
auc_vae =  0.7041485148514852
--------------------------------------------------
--------------------------------------------------
Эксперимент №50/56  с normal = Экономика-Госэкономика, anomal = Наука и техника-Оружие
auc_ae =  0.8247821782178218
auc_vae =  0.8247326732673268
--------------------------------------------------
--------------------------------------------------
Эксперимент №51/56  с normal = Экономика-Госэкономика, anomal = Культура-Театр
auc_ae =  0.8496237623762377
auc_vae =  0.8495940594059407
--------------------------------------------------
--------------------------------------------------
Эксперимент №52/56  с normal = Экономика-Госэкономика, anomal = Силовые структуры-Полиция и спецслужбы
auc_ae =  0.7615643564356437
auc_vae =  0.7616336633663368
--------------------------------------------------
--------------------------------------------------
Эксперимент №53/56  с normal = Экономика-Госэкономика, anomal = Спорт-Бокс и ММА
auc_ae =  0.4819207920792079
auc_vae =  0.4817128712871287
--------------------------------------------------
--------------------------------------------------
Эксперимент №54/56  с normal = Экономика-Госэкономика, anomal = Наука и техника-Космос
auc_ae =  0.7716336633663367
auc_vae =  0.7714752475247525
--------------------------------------------------
--------------------------------------------------
Эксперимент №55/56  с normal = Экономика-Госэкономика, anomal = Наука и техника-Наука
auc_ae =  0.9721386138613861
auc_vae =  0.9722475247524753
--------------------------------------------------
--------------------------------------------------
Эксперимент №56/56  с normal = Экономика-Госэкономика, anomal = Мир-Политика
auc_ae =  0.8626138613861386
auc_vae =  0.863
--------------------------------------------------

In [28]:
auc_list_ae = [
    0.903069306930693,
    0.8225445544554456,
    0.5366831683168317,
    0.7370594059405942,
    0.9738217821782178,
    0.9070396039603961,
    0.8140792079207921,
    0.6724257425742574,
    0.5414851485148515,
    0.2612970297029703,
    0.630831683168317,
    0.9162574257425743,
    0.7240792079207922,
    0.6346930693069308,
    0.7732178217821782,
    0.7854653465346534,
    0.4411881188118812,
    0.7571782178217822,
    0.9726237623762376,
    0.8266138613861386,
    0.7537425742574257,
    0.9394356435643565,
    0.9186732673267327,
    0.8899108910891089,
    0.9007623762376238,
    0.9926237623762376,
    0.953079207920792,
    0.9220693069306931,
    0.7428019801980199,
    0.8765544554455446,
    0.8035346534653465,
    0.5132277227722772,
    0.928009900990099,
    0.8967722772277228,
    0.7967227722772277,
    0.4378910891089109,
    0.6308712871287129,
    0.4983960396039604,
    0.17082178217821783,
    0.22625742574257426,
    0.6673861386138614,
    0.7532376237623762,
    0.7721683168316831,
    0.6600693069306931,
    0.3524059405940594,
    0.7291980198019802,
    0.9649009900990099,
    0.7044554455445544,
    0.8247821782178218,
    0.8496237623762377,
    0.7615643564356437,
    0.4819207920792079,
    0.7716336633663367,
    0.9721386138613861,
    0.8626138613861386,
]


auc_list_vae = [
    0.9030396039603961,
    0.8225049504950496,
    0.5369009900990099,
    0.7380000000000001,
    0.9738118811881188,
    0.907089108910891,
    0.814089108910891,
    0.6721683168316831,
    0.5413861386138614,
    0.2610891089108911,
    0.6314059405940595,
    0.9163564356435644,
    0.7239207920792079,
    0.6345049504950495,
    0.7731980198019802,
    0.7859405940594059,
    0.44102970297029703,
    0.7572772277227722,
    0.9726336633663366,
    0.8266336633663367,
    0.7538712871287129,
    0.9395445544554456,
    0.9186534653465347,
    0.8899801980198021,
    0.9007821782178218,
    0.9926237623762377,
    0.9530792079207921,
    0.9221881188118812,
    0.7424653465346535,
    0.8764455445544554,
    0.8037425742574258,
    0.5134158415841584,
    0.9279999999999999,
    0.8968118811881188,
    0.7963465346534654,
    0.7657029702970297,
    0.7310990099009901,
    0.4981881188118812,
    0.6683168316831682,
    0.7539306930693069,
    0.772128712871287,
    0.6598118811881187,
    0.35276237623762374,
    0.7292079207920793,
    0.9647821782178219,
    0.7041485148514852,
    0.8247326732673268,
    0.8495940594059407,
    0.7616336633663368,
    0.7817128712871287,
    0.7714752475247525,
    0.8722475247524753,
    0.863,
]

In [27]:
auc_np = np.array(auc_list_ae)
print("*" * 50)
print("AE Медиана auc = {}".format(np.median(auc_np)))
print("AE Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

auc_np = np.array(auc_list_vae)
print("*" * 50)
print("VAE Медиана auc = {}".format(np.median(auc_np)))
print("VAE Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

**************************************************
AE Медиана auc = 0.7721683168316831
AE Среднее auc = 0.7373074707470748
**************************************************
**************************************************
VAE Медиана auc = 0.7817128712871287
VAE Среднее auc = 0.7695359611432842
**************************************************
